In [41]:
import pandas as pd
import gensim
import logging


questions = pd.read_csv('data/2018-12-10-Treatd/LemmatizedQAFix.csv', sep=';', header=0)
terms = pd.read_csv('data/2018-12-10-Treatd/LemmatizedTerms.csv', sep=";")
termset = set(terms['term'])
termset.add('leite')
termset.add('boi')
termset.add('carrapato')

In [10]:
logging.basicConfig(format='%(asctime)s: %(levelname)s : %(message)s', level=logging.INFO)


In [42]:
def corpusreader(text):
    for i, row in text.iterrows():
        print(row['QuestionTerms'])
        if(i%10 == 0):
            logging.info("Read {0} reviews".format(i))
        mylist = row['QuestionTerms'].split()
        mylist.extend(row['AnswerTerms'].split())
        yield [w for w in mylist if w in termset]

documents = list(corpusreader(questions))
print(documents)
logging.info("Done reading data file")

2018-12-17 15:19:46,373: INFO : Read 0 reviews
2018-12-17 15:19:46,378: INFO : Read 10 reviews
2018-12-17 15:19:46,380: INFO : Read 20 reviews
2018-12-17 15:19:46,389: INFO : Read 30 reviews
2018-12-17 15:19:46,391: INFO : Read 40 reviews
2018-12-17 15:19:46,396: INFO : Read 50 reviews
2018-12-17 15:19:46,399: INFO : Read 60 reviews
2018-12-17 15:19:46,401: INFO : Read 70 reviews
2018-12-17 15:19:46,405: INFO : Read 80 reviews
2018-12-17 15:19:46,407: INFO : Read 90 reviews
2018-12-17 15:19:46,413: INFO : Read 100 reviews
2018-12-17 15:19:46,415: INFO : Read 110 reviews
2018-12-17 15:19:46,418: INFO : Read 120 reviews
2018-12-17 15:19:46,420: INFO : Read 130 reviews
2018-12-17 15:19:46,425: INFO : Read 140 reviews
2018-12-17 15:19:46,427: INFO : Read 150 reviews
2018-12-17 15:19:46,429: INFO : Read 160 reviews
2018-12-17 15:19:46,435: INFO : Read 170 reviews
2018-12-17 15:19:46,438: INFO : Read 180 reviews
2018-12-17 15:19:46,440: INFO : Read 190 reviews
2018-12-17 15:19:46,443: INFO :

1 quando iniciar o cuidado com o bezerro ? 
2 qual o principal cuidado que se dever tomar com o bezerro recém-nascer ? 
3 como se explicar o nascimento de bezerro fraco e pequeno ? 
4 qual o bom manejo de bezerro recém-nascer a fim de evitar o proliferação de doença ? 
5 qual o bom maneira de criar o bezerro : ao pé dar mãe ou apartar dela ? 
6 ser correto deixar para o bezerro apenas a “rapa de leite do quatro tetos” ( leite residual após o ordenha ) ? 
7 qual o vantagem do aleitamento natural ? qual o vantagem dar aleitamento artificial ? 
8 por que algum produtor fracassam ao mudar dar sistema de aleitamento natural para o artificial ? 
9 qual qualidade dever ter o pessoa que tratar do bezerro ? 
10 qual o causa dar mortalidade de bezerro no 3 primeiro mês de vida ? 
11 qual o vantagem de descornar o bezerro ainda jovem ? 
12 o leite “sujo” ser importante para o bezerro ? 
13 o colostro ser diferente do leitar normal ? seu composição variar após o parto ? 
14 haver diferença na prod

In [60]:
model = gensim.models.Word2Vec(documents, size = 5, window =10, min_count=2, workers=4)
model.train(documents, total_examples=len(documents), epochs=10)

2018-12-17 15:33:56,302: WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-12-17 15:33:56,303: INFO : collecting all words and their counts
2018-12-17 15:33:56,304: INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-17 15:33:56,307: INFO : collected 1609 word types from a corpus of 13808 raw words and 500 sentences
2018-12-17 15:33:56,308: INFO : Loading a fresh vocabulary
2018-12-17 15:33:56,310: INFO : effective_min_count=2 retains 981 unique words (60% of original 1609, drops 628)
2018-12-17 15:33:56,311: INFO : effective_min_count=2 leaves 13180 word corpus (95% of original 13808, drops 628)
2018-12-17 15:33:56,314: INFO : deleting the raw counts dictionary of 1609 items
2018-12-17 15:33:56,315: INFO : sample=0.001 downsamples 81 most-common words
2018-12-17 15:33:56,316: INFO : downsampling leaves estimated 9795 word corpus (74.3% of prior 13180)
2018-12-17 15:33:56,318: INFO : estimated required memory for 981 word

2018-12-17 15:33:56,520: INFO : worker thread finished; awaiting finish of 3 more threads
2018-12-17 15:33:56,522: INFO : worker thread finished; awaiting finish of 2 more threads
2018-12-17 15:33:56,527: INFO : worker thread finished; awaiting finish of 1 more threads
2018-12-17 15:33:56,528: INFO : worker thread finished; awaiting finish of 0 more threads
2018-12-17 15:33:56,528: INFO : EPOCH - 9 : training on 13808 raw words (9743 effective words) took 0.0s, 971239 effective words/s
2018-12-17 15:33:56,536: INFO : worker thread finished; awaiting finish of 3 more threads
2018-12-17 15:33:56,539: INFO : worker thread finished; awaiting finish of 2 more threads
2018-12-17 15:33:56,542: INFO : worker thread finished; awaiting finish of 1 more threads
2018-12-17 15:33:56,543: INFO : worker thread finished; awaiting finish of 0 more threads
2018-12-17 15:33:56,544: INFO : EPOCH - 10 : training on 13808 raw words (9822 effective words) took 0.0s, 924170 effective words/s
2018-12-17 15:33:

(97953, 138080)

In [72]:
relations = pd.DataFrame()
for term in termset:
    if term in model.wv.vocab:
        for otherTerm in model.wv.most_similar(positive=term):
            relations = relations.append({'Term1':term, 'Term2':otherTerm[0]}, ignore_index=True)
            #print(term, otherTerm[0])

In [74]:
print(relations.tail())

       Term1        Term2
9805  cabeça       demora
9806  cabeça     ovulação
9807  cabeça      líquida
9808  cabeça        parir
9809  cabeça  antibiótico


In [75]:
relations.to_csv('data/2018-12-17-Treatd/w2vrelations.csv')